This notebook provides guidance on how our popup "info" sheets for GAP species need to change to account for a whole new way of doing the Taxonomic Information Registry. These currently show up in the version 1 National Biogeographic Map behind the Biogeographic Analysis Package that presents information on species protection status by Ecoregion. Those were based on an older process involving items in ScienceBase that needs to completely go away. It is already stale, and some of the links on those items to BISON were not entirely accurate. The new info sheet will have sections for Taxonomic Authority information from ITIS, Federal Listing Status from ECOS TESS, and global/national/state status information from NatureServe.

All of this information is housed in the new Taxonomic Information Registry table in the GC2 instance. Appropriate attributes from the key/value pairs are pulled into a view, which populates an ElasticSearch index for use in our apps.

In [15]:
import requests
from IPython.display import display
apiBaseURL = "https://gc2.mapcentia.com/api/v1/sql/bcb"

### What's in the data?

First, it's important to understand the properties that are assembled in the Taxonomic Information Registry (TIR). The TIR is an evolving concept that will continue to be refined as we experiment with how all of this information is pulled together from various sources. The TIR stores all of its information in a PostgreSQL table with hstore type fields described below:

* registration - Variable pairs of information that represent the point of entry for taxa of interest to our data systems and analytical processes. This information is then used to kick off lookup lookup and matching processes that pull together other information of use.
* itis - Information returned from the Integrated Taxonomic Information System (ITIS) and cached for use as properties/variables in other systems. ITIS information changes routinely as taxonomic work is done by the editors, and so this "cache" will need to be refreshed through time. ITIS information is either retrieved by running a name search or by running a direct retrieval from a Taxonomic Serial Number from the registration information.
* worms - Similar to ITIS and coming soon, this is information from the World Register of Marine Species.
* tess - Information returned from the U.S. Fish and Wildlife Service Threatened and Endangered Species System on Federal listing status. TESS information is retrieved using the ITIS TSN, either from the original registration or from an ITIS match.
* natureserve - Information returned from the NatureServe species web services on global, national, and subnational (state) status of species populations. NatureServe information is either retrieved by a specified ID in the registration or by first running a species lookup to find an identifier.

The following section queries the TIR Property Registry and returns current definitions. We can use this source of definitions to return definitions whenever we need to explain one of the properties in the TIR.

In [12]:
r_properties = requests.get(apiBaseURL+"?q=SELECT * FROM tir.propertyregistry ORDER BY name").json()

for feature in r_properties["features"]:
    print(feature["properties"]["name"], " (", feature["properties"]["proptype"], ")")
    print(feature["properties"]["definition"])
    print("----")

acceptedTSN  ( integer )
Returned from ITIS, this is the accepted TSN for the taxon
----
cacheDate  ( datetime )
Date/time stamp generated and placed in any of the information pools to indicate when the information was retrieved and cached - may be found in multiple information sources
----
Class  ( string )
Taxonomic class from the taxonomic authority
----
CommonName  ( string )
For FWS TESS records, a common name for the taxon
----
createDate  ( date )
For ITIS records, the date that the taxonomic entry was created in ITIS
----
discoveredTSN  ( integer )
For ITIS records, the TSN that was discovered via name search
----
Division  ( string )
Taxonomic division from the taxonomic authority
----
DOI  ( string )
For GAP species registrations - the Digital Object Identifier assigned to the published habitat map
----
EGTID  ( integer )
The numeric part of a NatureServe identifier for a species
----
elementGlobalID  ( string )
NatureServe Element Global ID - used as a primary identifier acr

We can call up GAP species from the TIR in a number of ways. Since we have the GAP code as the primary identifier in most cases so far (e.g., the NBM BAPs), that's likely the hook that will be used most often. The following is a way to get this information via the GC2 SQL API.

In [16]:
GAP_SpeciesCode = "rSBLIx"
r_gapSpecies = requests.get(apiBaseURL+"?q=SELECT * FROM tir.tir2 WHERE registration->'GAP_SpeciesCode'='"+GAP_SpeciesCode+"'").json()
display(r_gapSpecies)

{'_execution_time': 0.096,
 'auth_check': {'auth_level': 'Write',
  'checked_relations': ['tir.tir2'],
  'session': None,
  'success': True},
 'features': [{'properties': {'gid': 19505,
    'itis': '"tsn"=>"564597", "rank"=>"Species", "Class"=>"Reptilia", "Genus"=>"Sceloporus", "Order"=>"Squamata", "usage"=>"valid", "Family"=>"Iguanidae", "Phylum"=>"Chordata", "Kingdom"=>"Animalia", "Species"=>"Sceloporus slevini", "Suborder"=>"Iguania", "nameWInd"=>"Sceloporus slevini", "Subfamily"=>"Phrynosomatinae", "Subphylum"=>"Vertebrata", "cacheDate"=>"2017-05-16T15:27:16.974388", "nameWOInd"=>"Sceloporus slevini", "Subkingdom"=>"Bilateria", "Superclass"=>"Tetrapoda", "createDate"=>"2000-01-31 17:38:44", "updateDate"=>"2005-07-29 00:00:00", "Infraphylum"=>"Gnathostomata", "Infrakingdom"=>"Deuterostomia", "itisMatchMethod"=>"TSNRetrival:564597", "vernacular:English"=>"Slevin\'\'s Bunchgrass Lizard"',
    'natureserve': '"cacheDate"=>"2017-05-22T19:51:23.572217", "elementGlobalID"=>"ELEMENT_GLOBAL

This is really a PostGIS return, so it has a lot of extra stuff. We may eventually need to work up some other way of storing these data, but for now, we'll generate a view with the specific attributes we want and throw that into ElasticSearch. I created a new "gapspecies" schema in the GC2 instance and will start building out what we want to work with there.